In [3]:
from pandasql import sqldf
import pandas as pd

In [4]:
events = pd.read_csv('events.csv')

In [5]:
query = '''
/* Создадим таблицу в которую запишем информацию о текущем и предыдущем событиях для данного юзера */

WITH SessionData AS (
    SELECT
        user_id,
        event_dtm,
        event_name,
        LAG(event_name) OVER (PARTITION BY user_id ORDER BY event_dtm) AS previous_event,
        LAG(event_dtm) OVER (PARTITION BY user_id ORDER BY event_dtm) AS previous_event_dtm
    FROM events
),

/* Далее, найдём первые и последние события в сессиях с учётом условий, обозначенных в формулировке задачи */

SessionStarts AS (
    SELECT
        user_id,
        event_dtm AS session_start_dtm,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_dtm) AS session_number
    FROM SessionData
    WHERE event_name = 'session_start'
        OR previous_event = 'session_end'
        OR previous_event IS NULL
        OR (- strftime('%s', previous_event_dtm) + strftime('%s', event_dtm)) / 60 > 30
        OR DATE(previous_event_dtm) <> DATE(event_dtm )
),

SessionEnds AS (
    SELECT
        user_id,
        previous_event_dtm AS session_end_dtm,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_dtm) AS session_number
    FROM SessionData
    WHERE previous_event = 'session_end'
        OR event_name = 'session_start'
        OR (- strftime('%s', previous_event_dtm) + strftime('%s', event_dtm)) / 60 > 30
        OR DATE(previous_event_dtm) <> DATE(event_dtm)
),

/* Найдём всех зарегистрированных юзеров - таких у которых в истории есть событие "purchase" */

Clients AS (
    SELECT DISTINCT
        user_id
    FROM SessionData
    WHERE event_name = 'purchase'
)

/* Соберём всё в один запрос. В качестве session_id можно взять просто пронумерованные сессии */

SELECT
    ROW_NUMBER() OVER() - 1 AS session_id,
    ss.user_id,
    ss.session_start_dtm,
    se.session_end_dtm,
    ss.session_number,
    CASE WHEN Clients.user_id IS NOT NULL THEN 1 ELSE 0 END AS is_client
    /* Так как мы присоединили таблицу с клиентами левым джоином, напротив у тех клиентов которых нет в таблице Clients
    поле Clients.user_id будет равно NULL */
FROM SessionStarts ss
JOIN SessionEnds se ON ss.user_id = se.user_id AND ss.session_number = se.session_number
LEFT JOIN Clients ON ss.user_id = Clients.user_id
ORDER BY ss.user_id, ss.session_start_dtm;
'''

result = sqldf(query)